# Построение модели сентимент анализа отзывов на телефоны.
## Выборка была собрана в предыдущем ноутбуке.

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from collections import Counter
warnings.filterwarnings('ignore')

## Считаем данные из файлов и посморим на них

In [2]:
with open('test.csv', 'r') as f:
    text = f.read()
    text = text.replace('\n', ' ')
    komms = [rev.strip() for rev in re.findall(r'<review>(.+?)</review>', text)]
    pd.DataFrame(data={'komment': komms}).to_csv('test_data.csv', index=False)

In [3]:
df = pd.read_csv('revs.csv')
data = pd.read_csv('test_data.csv')

In [4]:
data.head()

,komment
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях К сожалению, факт пол..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


In [5]:
df.head()

,pos,neg,komment,result
0,"Наклеена защитная пленка,чехол из коробки ,дов...",Предустановленные приложения производителя кот...,Полностью соответствует моим потребностям!Особ...,Отличный
1,1. Хороший дизайн2. Камера довольно высокого у...,"Экран поддерживает только частоту в 60 Гц, в о...",В целом это лучший вариант на мой взгляд в дан...,Отличный
2,В целом хороший телефон. Просто мне не повезло.,Фронталка мутно фоткала. Сравнила с выставочны...,Заметила косяк фронтальной камеры. В отличие о...,Обычный
3,"Очень шустрый, с хорошим железом и камерами. П...",NaN,Однозначно рекомендую брать,Отличный
4,Долго держит батарею.Камера.Возможность встави...,Плохо снимает видео.Нужно привыкать к Амолед э...,"С 2014 по 2017 пользовался только Сяоми, так к...",Хороший


In [6]:
df.result.value_counts()

Отличный    1341
Хороший      273
Обычный      152
Ужасный      127
Плохой       106
Хорош          1
Name: result, dtype: int64

## Так как в форме яндекс маркета есть три поля: достоинства, недостатки и комментарий, пользователи часто заполняют поле недостатков на отличные товары словами: 'нет', 'не найдены' и тд. Тоже самое часто бывает в достоинствах ужасных товаров. Удалим такие слова, чтобы они не сбивали модель.

In [7]:
list(df[df.result.isin(['Отличный'])].neg)

['Предустановленные приложения производителя которые и не нужны но и удалить их нельзя',
 'Экран поддерживает только частоту в 60 Гц, в отличии от версии Pro.Иногда снимки получаются размазанными. Фокусировка при движении не идеальна.',
 nan,
 'Не найдены,либо незначительные',
 'Нет',
 'Не обнаружила',
 'Нет.',
 'Нет.',
 nan,
 'привезли не тот цвет !!!',
 nan,
 'как мне казалось зарядку жрёт быстрее чем в своё время новый редми ноут 5 про 4/64, но возможно мне это только кажется))) на два дня с соцсетями, фотографированием по работе мне хватает',
 'Андроид. ))))Есть замечание - динамик! Ну просто все слышат, что мне отвечает собеседник. Правда, странный прикол.',
 nan,
 'Пока не заметили',
 'Иногда подвисает',
 'Нет лампочки индикатора пропущенного события',
 'Пока все понравилось,',
 nan,
 'не совсем понятный интерфейс, после установки мелодии вызова через несколько часов на любой входящий вызов вместо установленой мелодии еле слышное пиканье, в источнике проблемы пока не разобрался -

In [8]:
list(df[df.result.isin(['Хороший'])].neg)

['Плохо снимает видео.Нужно привыкать к Амолед экрану.Нелогичное и частями сложное меню.Походу с клонами контактов Сяоми так и не решил проблему, а уже 4 года прошло.',
 'Камера, нет индикатора сообщений, always on display активен только 10 секунд',
 'Камера!!!',
 'Экран (точнее наклейка убивается от чиха на него) , размер тк если вы не пионист то вам тяжело будет дотянуться до места вырубающего то или иное приложение (свайп во многих не робит) , камера шлак (пока она сфокусируется у тебя появиться семья и ты поймешь что фото в движении эта камера просто не берет)',
 'Реклама в операционке, благо есть инструкции на ютубе как ее вырезать',
 'Не понравилась камера через родное приложение. Фотографии часто пересвечены, есть эффект масляной картинки какой-то. Ожидал, что будет лучше. Может исправят в дальнейшем. Пользуюсь gcam, через него получается лучше.',
 'спустя неделю начал хрипеть динамик при разговоре на громкой связи',
 'Именно для жены минусом является наличие сканера отпечатков 

In [9]:
list(df[df.result.isin(['Обычный'])].pos)

['В целом хороший телефон. Просто мне не повезло.',
 'ЦенаХорошо держит заряд при умеренном использовании',
 'Экран супер, камера супер',
 nan,
 'Быстрая доставка. Меньше недели до Самары.Телефон классный, жена довольна.',
 'В целом неплохой вариант, но не за такие деньги',
 nan,
 'Обычный.',
 'Неплохой экран',
 'Маленький, громкий звук кнопок',
 'Аккумулятор депжит зарядку смело дней на 5',
 'Цена, упаковка',
 nan,
 'Крепкий, яркий фонарик',
 nan,
 'Цена',
 nan,
 nan,
 'отсутствие камеры, 2 симки',
 'Цена,компактность,фонарик нормально светит.',
 'Дешево, ничего лишнего, если брать для звонков, фонарик',
 'Для моих целей он меня устраивает.',
 'Цена',
 'Цена',
 'Цена. Уровень зарядки.',
 'Цена кайф, даже с учетом падающего рубля, урвал на маркете за 5к (с учетом баллов плюса). Огромный экран, довольно шустрый. В целом, это все, что могу сказать хорошего).',
 nan,
 'Айфон.',
 'Легкий, amoled экран хорош.',
 'Новый HDR дисплейСпорно - камера',
 'Айфон',
 '1. Лаконичный дизайн.2. Хорошо 

In [10]:
list(df[df.result.isin(['Плохой'])].pos)

['Приятный дизайн, экран насыщенный, легкий, идеальный размер',
 'Обычная звонилка',
 'Цена, очень лёгкий',
 'Принимает звонки и смс, и отправляет. Есть радио и фонарик. Стоит ерунду.',
 'Копия НОКИА 105. Звук хороший.',
 'Дизайн',
 'Компактный и прост в использовании',
 'легкий',
 'Цена? Хотя зачем платить за бесполезную вещь?',
 'Цена',
 'дешевый обычный телефон, поразило что до сих пор есть телефоны с отдельной батарейкой',
 'Цена',
 'Работает',
 'За эти деньги вполне быстрый, пока система чистая конечно',
 nan,
 'Их нет, от Нокии остались только буквы',
 'красивый, удобный, боле-мене приличная камера',
 'Издалека смотрится симпатично',
 'Выглядит хорошо.',
 nan,
 'В телефоне работает симка оператора TELE2',
 'Дизайн.WhatsApp.',
 'Быстро, чётко , что заказал , то и получил .Плюсы? несколько ; аккумулятор, долго держит, если не смотреть YouTube дня 3,4NFC высший пилотаж, на расстоянии см 25 уже срабатывает и быстроОтпечаток пальца удобно расположен.Быстрая зарядка , кабель tape-c!',


In [11]:
list(df[df.result.isin(['Ужасный'])].pos)

['Нет',
 'Нет',
 nan,
 nan,
 'Лёгкие, удобный',
 'Нет',
 'Никаких',
 '-',
 nan,
 'Нет',
 'Обычный телефон',
 'Нет достоинств',
 'Нет',
 'Неплохой дисплей, нормальный мелодии',
 nan,
 'красивый дизайн и цена',
 nan,
 'Нет',
 'Нет',
 nan,
 'цена в теории если бы он работал',
 nan,
 'Достоинств нет.',
 nan,
 'цена',
 nan,
 'нет',
 nan,
 'Конструкция, функции- супер.',
 nan,
 'Раздача интернета',
 'Хороший звук через динамики и микрофон.',
 'Нет',
 'Нет достоинств! Самая неудачная модель во всей линейке Нокиа!Мне стыдно за эту компанию и за тех неумелых разработчиков, которыеполностью провалили (на программном и аппаратном уровне) данную модель!',
 'внешний вид - и всё!',
 'Кнопочные, хороший микрофон и динамик',
 'Очень красив внешний вид.',
 'Только внешний вид',
 'Только внешний вид.',
 'Пользовался сутки…потом терпение закончилось.',
 'Нет',
 'нет никаких достоинств',
 'Ни чего не понравилось',
 nan,
 'Дизайн',
 nan,
 'Ничего особенного, что было в прежних моделях. Разве что можно боле

## Обрежем слишком короткий текст из поля 'достоинства' в ужасных товарах и поля 'недостатки' отличных товаров. Подразумевается, что более длинные отзывы все таки содержат полезную информацию.

In [12]:
df.fillna('', inplace=True)

In [13]:
df['target'] = df['result'].map({'Отличный': 1, 'Хороший': 1, 'Плохой': 0, 'Ужасный': 0})

In [14]:
df.loc[(df.result == 'Отличный') & (df.neg.map(len) < 20), 'neg'] = ''
df.loc[(df.result == 'Ужасный') & (df.pos.map(len) < 20), 'pos'] = ''

## Проверим три гипотезы:
## 1. Возьмем полный тест отзыва из достоинств, недостатков и комментария.
## 2. Возможно люди не стали бы указывать столько недостатков в положительных отзывах и достоинств в отрицательных, если бы не форма яндекса, возьмем положительные отзывы без поля недостатков и отрицательные без достоинств, получим более однозначно окрашенный текст.
## 3. Возьмем достоинства как положительные отзывы, недостатки как отрицательные.

In [15]:
df['full_text'] = (df.pos + [' ']*len(df) + df.neg + [' ']*len(df) + df.komment)

In [16]:
df['sentiment_text'] = (df[df.target == 1].pos + [' ']*len(df[df.target == 1]) + df[df.target == 1].komment)
df.loc[df.target == 0, 'sentiment_text'] = (df[df.target == 0].neg + 
                                            [' ']*len(df[df.target == 0]) + df[df.target == 0].komment)

In [17]:
df.head()

,pos,neg,komment,result,target,full_text,sentiment_text
0,"Наклеена защитная пленка,чехол из коробки ,дов...",Предустановленные приложения производителя кот...,Полностью соответствует моим потребностям!Особ...,Отличный,1.0,"Наклеена защитная пленка,чехол из коробки ,дов...","Наклеена защитная пленка,чехол из коробки ,дов..."
1,1. Хороший дизайн2. Камера довольно высокого у...,"Экран поддерживает только частоту в 60 Гц, в о...",В целом это лучший вариант на мой взгляд в дан...,Отличный,1.0,1. Хороший дизайн2. Камера довольно высокого у...,1. Хороший дизайн2. Камера довольно высокого у...
2,В целом хороший телефон. Просто мне не повезло.,Фронталка мутно фоткала. Сравнила с выставочны...,Заметила косяк фронтальной камеры. В отличие о...,Обычный,NaN,В целом хороший телефон. Просто мне не повезло...,NaN
3,"Очень шустрый, с хорошим железом и камерами. П...",,Однозначно рекомендую брать,Отличный,1.0,"Очень шустрый, с хорошим железом и камерами. П...","Очень шустрый, с хорошим железом и камерами. П..."
4,Долго держит батарею.Камера.Возможность встави...,Плохо снимает видео.Нужно привыкать к Амолед э...,"С 2014 по 2017 пользовался только Сяоми, так к...",Хороший,1.0,Долго держит батарею.Камера.Возможность встави...,Долго держит батарею.Камера.Возможность встави...


## Моделирование.

In [18]:
def make_pipeline(vectorizer, classifier):
    return Pipeline([('vectorizer', vectorizer), ('classifier', classifier)])

In [19]:
def clf_results(X, y):
    for key, clf in {'LinearSVC': LinearSVC, 'LogisticRegression': LogisticRegression,
                    'RidgeClassifier': RidgeClassifier, 'SGDClassifier': SGDClassifier,
                    'DecisionTreeClassifier': DecisionTreeClassifier, 
                    'GradientBoostingClassifier': GradientBoostingClassifier,
                    'RandomForestClassifier': RandomForestClassifier}.items():
        score = cross_val_score(make_pipeline(TfidfVectorizer(), clf(random_state=1)), X, y, cv=5).mean()
        print(f"{key} - {score}")

    for key, clf in {'MultinomialNB': MultinomialNB, 'BernoulliNB': BernoulliNB}.items():
        score = cross_val_score(make_pipeline(TfidfVectorizer(), clf()), X, y, cv=5).mean()
        print(f"{key} - {score}")  

In [20]:
def make_predictions(clf, df, X_test):
    try:
        pipe = make_pipeline(TfidfVectorizer(), clf(random_state=1))
    except:
        pipe = make_pipeline(TfidfVectorizer(), clf())
    pipe.fit(df.iloc[:, 0], df.iloc[:, 1])
    res = pipe.predict(X_test)
    return res

In [21]:
def save_predictions(preds):
    res = pd.DataFrame(data=preds, columns=['y'])
    res.index.name = 'Id'
    res['y'] = res.y.map({0: 'neg', 1: 'pos'})
    res.to_csv('res.csv')    

## Балансируем классы.

In [22]:
print('Положительных:', sum(df.target == 1),'\nОтрицательных:' , sum(df.target == 0))

Положительных: 1614 
Отрицательных: 233


In [23]:
random.seed(1)
d = {'full_text': random.choices(list(df[df.target == 0].full_text), 
                                  k=(sum(df.target == 1) - sum(df.target == 0))), 
     'target': np.zeros(sum(df.target == 1) - sum(df.target == 0))}
new_df = pd.DataFrame(data=d)

In [24]:
full_df = pd.concat([df[['full_text', 'target']].dropna(), new_df])
full_df.target.value_counts()

0.0    1614
1.0    1614
Name: target, dtype: int64

## Смотрим качество классификаторов с помощью кроссвалидации на собранной выборке и полном тексте отзывов.

In [25]:
clf_results(full_df.full_text, full_df.target)

LinearSVC - 0.9745861233110137
LogisticRegression - 0.9429692562459501
RidgeClassifier - 0.9652876376988985
SGDClassifier - 0.977375380996952
DecisionTreeClassifier - 0.9284109727122182
GradientBoostingClassifier - 0.8810089519283846
RandomForestClassifier - 0.9872911416708666
MultinomialNB - 0.8949494804041567
BernoulliNB - 0.7948765209878321


In [26]:
save_predictions(make_predictions(RandomForestClassifier, full_df, data.komment))

## Получаем на каггле Score: 0.58888. Это намного хуже, чем на своей выборке.
## Попробуем второй по accuracy классификатор.

In [27]:
save_predictions(make_predictions(SGDClassifier, full_df, data.komment))

## Score: 0.72222. Результат стал лучше, но все равно на других отзывах тональность предсказывается плохо, попробуем другую гипотезу.

## Балансируем классы.

In [28]:
random.seed(1)
d = {'sentiment_text': random.choices(list(df[df.target == 0].sentiment_text), 
                                  k=(sum(df.target == 1) - sum(df.target == 0))), 
     'target': np.zeros(sum(df.target == 1) - sum(df.target == 0))}
new_df = pd.DataFrame(data=d)

In [29]:
sentiment_df = pd.concat([df[['sentiment_text', 'target']].dropna(), new_df])
sentiment_df.target.value_counts()

0.0    1614
1.0    1614
Name: target, dtype: int64

## Обучим классификаторы на однозначно окрашенном тексте.

In [30]:
clf_results(sentiment_df.sentiment_text, sentiment_df.target)

LinearSVC - 0.9659130726954185
LogisticRegression - 0.9398742410060719
RidgeClassifier - 0.9618801449588403
SGDClassifier - 0.9708695130438956
DecisionTreeClassifier - 0.9206897544819641
GradientBoostingClassifier - 0.8989852881176951
RandomForestClassifier - 0.9686984904120768
MultinomialNB - 0.9225252597979215
BernoulliNB - 0.7945736434108527


In [31]:
save_predictions(make_predictions(SGDClassifier, sentiment_df, data.komment))

## Score: 0.82222. Качество еще улучшилось, попробуем третью гипотезу.

## Балансируем классы.

In [32]:
df.replace('', np.nan, inplace=True)

In [33]:
print('Положительных:', len(df.pos.dropna()),'\nОтрицательных:' , len(df.neg.dropna()))

Положительных: 1779 
Отрицательных: 1095


In [34]:
d = {'pos_neg_text': pd.concat([df.pos.dropna(), df.neg.dropna()]), 
     'target': [1]*len(df.pos.dropna()) + [0]*len(df.neg.dropna())}
etc_df = pd.DataFrame(data=d)

In [35]:
random.seed(1)
d = {'pos_neg_text': random.choices(list(df.neg.dropna()), 
                                  k=(len(df.pos.dropna()) - len(df.neg.dropna()))), 
     'target': np.zeros(len(df.pos.dropna()) - len(df.neg.dropna()))}
new_df = pd.DataFrame(data=d)

In [36]:
pos_neg_df = pd.concat([etc_df, new_df])
pos_neg_df.target.value_counts()

0.0    1779
1.0    1779
Name: target, dtype: int64

## Обучим классификаторы на достоинстах и недостатках.

In [37]:
clf_results(pos_neg_df.pos_neg_text, pos_neg_df.target)

LinearSVC - 0.9052880892555193
LogisticRegression - 0.8839168602537967
RidgeClassifier - 0.8979764218777161
SGDClassifier - 0.9019188830417674
DecisionTreeClassifier - 0.8636925362284487
GradientBoostingClassifier - 0.8265842538598903
RandomForestClassifier - 0.8870063528184705
MultinomialNB - 0.8887000426681837
BernoulliNB - 0.6987045465320249


In [38]:
save_predictions(make_predictions(SGDClassifier, pos_neg_df, data.komment))

## Score: 0.92222. Результат уже удовлетворительный и не отличается от скора на тренировочной выборке.